In [1]:
import numpy as np
import re
import spacy
import sys
from spacy.lang.en import English
from zipfile import ZipFile

maxSeqLen = 500
maxTotLen = maxSeqLen + (maxSeqLen - 1)

tokenIndex = dict()
index = 3
for line in open("vocabulary.dat", "r", encoding = "utf-8"):
    value = line.strip("\r\n").split("\t")
    tokenIndex[value[1]] = index
    index = index + 1

vocabularySize = index

nlp = spacy.load("en_core_web_sm")
tokenizer = English().Defaults.create_tokenizer(nlp)

def getNgrams(sequence, n = 2):
    return set(zip(*[sequence[i:] for i in range(n)]))

gramIndex = dict()
missingGramIndex = index
nextGramIndex = index + 1
for partition in [ "trn", "val", "tst" ]:
    with ZipFile("imdb_" + partition + ".zip", "r") as zipFile:
        docCount = len(zipFile.infolist())
        docTokenMatrix = np.zeros((docCount, maxTotLen + 1)).astype("int32")
        for i in range(docCount):
            with zipFile.open(str(i).zfill(5) + ".txt") as file:
                text = file.read().decode("utf-8")
                text = re.sub("[\t\r\n]", " ", text)
                text = " ".join(text.split())
                sequence = []
                seqLen = 0
                for token in [ token.text for token in tokenizer(text) ]:
                    key = token.lower()
                    if (seqLen < maxSeqLen):
                        if (key in tokenIndex):
                            sequence.append(tokenIndex[key])
                        else:
                            sequence.append(2)
                        seqLen = seqLen + 1
                gramList = getNgrams(sequence, n = 2)
                for gram in gramList:
                    if (gram not in gramIndex) and (partition == "trn"):
                        gramIndex[gram] = nextGramIndex
                        nextGramIndex = nextGramIndex + 1
                    if (seqLen < maxTotLen):
                        if (gram in gramIndex):
                            sequence.append(gramIndex[gram])
                        else:
                            sequence.append(missingGramIndex)
                        seqLen = seqLen + 1
                padLen = maxTotLen + 1 - seqLen - 1
                colIndex = 0
                for j in range(padLen):
                    docTokenMatrix[i,colIndex] = 0
                    colIndex = colIndex + 1
                docTokenMatrix[i,colIndex] = 1
                colIndex = colIndex + 1
                for k in sequence:
                    docTokenMatrix[i,colIndex] = k
                    colIndex = colIndex + 1
        np.save("imdb_" + partition + "X.npy", docTokenMatrix)